# Foreign Key Processing

In [2]:
# ! py -m pip install networkx

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.6 MB 1.7 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.6 MB 2.9 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.6 MB 4.3 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.6 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import networkx as nx

In [5]:
path = 'C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Security\\3. Data and Modeling\\Data Dictionary\\IFACTS\\' #path to where files for the database are kept
database = 'IFACTS'
date = '20240515'

In [6]:
raw_FKs = pd.read_excel(path+database+'_FKs_'+date+'.xlsx')

## Manual Analysis

In [ ]:

starting_table = 'datRequest_ForeignNational'

In [7]:
raw_FKs#['TableName'].value_counts()

,SchemaName,TableName,ReferencedTable,ColumnName,FK_name,ReferencedColumn
0,dbo,aspnet_Membership,aspnet_Applications,ApplicationId,FK__aspnet_Me__Appli__1FD8A9E3,ApplicationId
1,dbo,aspnet_Membership,aspnet_Users,UserId,FK__aspnet_Me__UserI__20CCCE1C,UserId
2,dbo,aspnet_Paths,aspnet_Applications,ApplicationId,FK__aspnet_Pa__Appli__51700577,ApplicationId
3,dbo,aspnet_PersonalizationAllUsers,aspnet_Paths,PathId,FK__aspnet_Pe__PathI__5728DECD,PathId
4,dbo,aspnet_PersonalizationPerUser,aspnet_Paths,PathId,FK__aspnet_Pe__PathI__5AF96FB1,PathId
...,...,...,...,...,...,...
111,dbo,datUserReportCol,datUserReport,UserReport_ID,FK_datUserReportCol_datUserReport,UserReport_ID
112,dbo,datUserReportUdf,datReportUdf,ReportUdf_ID,FK_datUserReportUdf_datReportUdf,ReportUdf_ID
113,dbo,datUserReportUdf,datUserReport,UserReport_ID,FK_datUserReportUdf_datUserReport,UserReport_ID
114,dbo,luCountry,luCountryType,CountryType_ID,FK_luCountry_luCountryType,CountryType_ID


In [9]:
raw_FKs[raw_FKs['TableName']==starting_table]

,SchemaName,TableName,ReferencedTable,ColumnName,FK_name,ReferencedColumn
51,dbo,datRequest_ForeignNational,datApprRequestForm,ApprRequestForm_ID,FK_datRequest_ForeignNational_datApprRequestForm,ApprRequestForm_ID
52,dbo,datRequest_ForeignNational,luCountry,BirthCitiz_ID,FK_datRequest_ForeignNational_luCountry1,Country_ID
53,dbo,datRequest_ForeignNational,luCountry,BirthCountry_ID,FK_datRequest_ForeignNational_luCountry,Country_ID
54,dbo,datRequest_ForeignNational,luCountry,CountryResidence_ID,FK_datRequest_ForeignNational_luCountry4,Country_ID
55,dbo,datRequest_ForeignNational,luCountry,DualCitiz1_ID,FK_datRequest_ForeignNational_luCountry2,Country_ID
56,dbo,datRequest_ForeignNational,luCountry,DualCitiz2_ID,FK_datRequest_ForeignNational_luCountry3,Country_ID
57,dbo,datRequest_ForeignNational,luCountry,EmployerCountry_ID,FK_datRequest_ForeignNational_luCountry6,Country_ID
58,dbo,datRequest_ForeignNational,luSupportingDocu,I94SuppDoc_ID,FK_datRequest_ForeignNational_luSupportingDocu,SupportingDocu_ID
59,dbo,datRequest_ForeignNational,luCountry,PassportCountry_ID,FK_datRequest_ForeignNational_luCountry5,Country_ID
60,dbo,datRequest_ForeignNational,luVisaCode,VisaCode_ID,FK_datRequest_ForeignNational_luVisaCode,VisaCode_ID


In [20]:
fks = raw_FKs.groupby('TableName').agg({'ReferencedTable': lambda x: x.tolist()}).reset_index()

In [21]:
rev_fks = raw_FKs.groupby('ReferencedTable').agg({'TableName': lambda x: x.tolist()}).reset_index()

In [22]:
fks_dict = pd.Series(fks.ReferencedTable.values,index=fks.TableName).to_dict()

In [23]:
rev_fks_dict = pd.Series(rev_fks.TableName.values,index=rev_fks.ReferencedTable).to_dict()

In [29]:
fks_dict['datRequest_ForeignNational']

['datApprRequestForm',
 'luCountry',
 'luCountry',
 'luCountry',
 'luCountry',
 'luCountry',
 'luCountry',
 'luSupportingDocu',
 'luCountry',
 'luVisaCode']

In [24]:
rev_fks_dict['datRequest_ForeignNational']

['datRequest_FN_FamilyMember']

In [25]:
def create_list_from_table (dict, rev_dict, list):
    if len(list) == 0:
        return
    else:
        for table in list:
            if table in dict:                
                sublist = dict[table]
                # print('Tables referenced by ', table,':')           
                for item in sublist:    
                    print(item)
                create_list_from_table(dict, rev_dict, sublist)
            if table in rev_dict:
                revsublist = rev_dict[table]
                # print('Tables that reference ', table, ':')
                for item in revsublist:
                    print(item)
            #     create_list_from_table(dict, rev_dict, revsublist) # infinite loop


In [26]:
create_list_from_table(fks_dict, rev_fks_dict, ['datRequest_ForeignNational'])

datApprRequestForm
luCountry
luCountry
luCountry
luCountry
luCountry
luCountry
luSupportingDocu
luCountry
luVisaCode
luVisitStatus
luCompany
luExpeditedRequestState
luInteractionType
luInteractionType
luPurposeCode
luSensitiveTopic
luState
datApprRequestForm
datREVApprRequestForm
datApprRequestForm
datREVApprRequestForm
datApprRequestForm
datREVApprRequestForm
datApprRequestForm
datApprRequestForm
datREVApprRequestForm
datREVApprRequestForm
datApprRequestForm
datApprRequestForm
datREVApprRequestForm
datREVApprRequestForm
datApprRequestForm
datREVApprRequestForm
datApprRequestForm
datREVApprRequestForm
datApprRequestForm
datREVApprRequestForm
brRequest_Area
brRequest_Software
brRequest_StateAction
brRequest_SubjectCode
datAddtSubmitter
datAwareIndividual
datEscort
datHistoricalSecPlan
datPrevisitBriefingLog
datRequest_ForeignNational
datRequest_OpenConcur
datRequestComputer
datRequestSecArea
datSecPlanLetter
datSTMatrixAffiliate
datSTMatrixLived
datSTMatrixRelationship
datSTMatrixVisit


In [42]:
listoftables = pd.read_clipboard(header=None,names=['TableName'])

In [50]:
listoftables['TableName'].nunique()

34

## Analysis with NetworkX

Documentation: https://networkx.org/documentation/stable/reference/index.html

In [9]:
edgelist = raw_FKs[['TableName', 'ReferencedTable']]

In [10]:
edgelist

,TableName,ReferencedTable
0,aspnet_Membership,aspnet_Applications
1,aspnet_Membership,aspnet_Users
2,aspnet_Paths,aspnet_Applications
3,aspnet_PersonalizationAllUsers,aspnet_Paths
4,aspnet_PersonalizationPerUser,aspnet_Paths
...,...,...
111,datUserReportCol,datUserReport
112,datUserReportUdf,datReportUdf
113,datUserReportUdf,datUserReport
114,luCountry,luCountryType


In [11]:
edgelist.to_csv(database+'_edgelist.csv', index=False, header=False)

In [26]:
fk_graph = nx.read_edgelist(database+'_edgelist.csv', delimiter=',',create_using=nx.DiGraph)

In [27]:
fk_graph.number_of_nodes()

83

In [28]:
fk_graph.number_of_edges()

100

In [47]:
nx.is_directed_acyclic_graph(fk_graph)

True

In [30]:
nx.is_weakly_connected(fk_graph)

False

In [32]:
nx.number_weakly_connected_components(fk_graph)

5

In [40]:
for c in nx.weakly_connected_components(fk_graph):
    print(len(c), ':', c)

9 : {'aspnet_Membership', 'aspnet_Paths', 'aspnet_Roles', 'aspnet_PersonalizationPerUser', 'aspnet_UsersInRoles', 'aspnet_PersonalizationAllUsers', 'aspnet_Users', 'aspnet_Profile', 'aspnet_Applications'}
62 : {'datREVRequest_ForeignNational', 'datREVHistoricalSecPlan', 'datPrevisitBriefingLog', 'datREVSTMatrixFinalAppr', 'enumHistoricalDataType', 'datRequest_OpenConcur', 'luSoftware', 'luSoftwareType', 'datAwareIndividual', 'datRequest_ForeignNational', 'datREVApprRequestForm', 'datStateActionLog', 'datRequestComputer', 'datRequest_FN_FamilyMember', 'brRequest_StateAction', 'datREVRequestSecArea', 'datHistoricalSecPlan', 'luRelationship', 'datREVSTMatrixVisit', 'datRequestSecArea', 'datREVAddtSubmitter', 'luCountryType', 'brREVRequest_SubjectCode', 'datREVSTMatrixLived', 'luAreaType', 'datSTMatrixVisit', 'datREVRequestComputer', 'brRequest_Area', 'brREVRequest_Area', 'datREVSTMatrixRelationship', 'luExpeditedRequestState', 'luVisaCode', 'luCountry', 'datArea', 'datREVSTMatrixAffiliate